In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter
import optuna as op

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
from libs.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from libs.Evaluation.Evaluator import EvaluatorHoldout

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
slim_res = pd.read_csv("result_experiments/SLIM/results.csv")

In [10]:
slim_res.sort_values(by=['RECALL', "MAP"], ascending=False)

,topK,l1_ratio,alpha,time,cutoff,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
75,330,0.11625,0.00026,2024-12-01 11:03:56,10,0.111503,0.166348,0.141143,0.060451,0.087932,...,0.994991,0.569510,0.994991,0.239853,13.448351,0.999872,0.394507,0.911093,1.367301,0.243768
55,320,0.11625,0.00026,2024-12-01 04:02:42,10,0.111522,0.166355,0.141133,0.060464,0.087947,...,0.994991,0.569510,0.994991,0.240054,13.450461,0.999873,0.394839,0.911236,1.364541,0.243797
103,350,0.11100,0.00026,2024-12-02 04:07:47,10,0.111497,0.166318,0.141127,0.060441,0.087900,...,0.994991,0.569314,0.994991,0.240487,13.451387,0.999872,0.395552,0.911299,1.368347,0.243777
63,330,0.11100,0.00026,2024-12-01 06:51:24,10,0.111536,0.166336,0.141116,0.060466,0.087930,...,0.994991,0.569398,0.994991,0.240864,13.455295,0.999873,0.396171,0.911564,1.363084,0.243832
79,330,0.11800,0.00026,2024-12-01 13:17:13,10,0.111480,0.166309,0.141116,0.060447,0.087918,...,0.994991,0.569482,0.994991,0.239611,13.446661,0.999872,0.394110,0.910979,1.368187,0.243756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,320,0.11100,0.00020,2024-11-30 22:44:29,10,0.111103,0.165689,0.140637,0.060292,0.087724,...,0.994991,0.568223,0.994991,0.254819,13.542520,0.999888,0.419124,0.917473,1.316684,0.244639
80,340,0.11100,0.00020,2024-12-01 13:39:54,10,0.111162,0.165720,0.140635,0.060310,0.087736,...,0.994991,0.568306,0.994991,0.254164,13.537005,0.999887,0.418047,0.917100,1.323202,0.244562
140,370,0.11100,0.00020,2024-12-03 10:19:45,10,0.111148,0.165709,0.140625,0.060311,0.087738,...,0.994991,0.568278,0.994991,0.253398,13.529806,0.999884,0.416786,0.916612,1.333071,0.244459
20,310,0.11100,0.00020,2024-11-30 08:15:55,10,0.111084,0.165681,0.140623,0.060299,0.087732,...,0.994991,0.568027,0.994991,0.255103,13.545145,0.999889,0.419591,0.917651,1.312987,0.244680


In [5]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [6]:
URM_train = sps.load_npz("URM_train_ens.npz")
URM_val = sps.load_npz("URM_val_ens.npz")

In [7]:
#URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [8]:
slim_model = SLIMElasticNetRecommender

In [16]:
evaluator_validation = EvaluatorHoldout(URM_val, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1008 ( 2.8%) Users that have less than 1 test interactions


In [26]:
def objective(optuna_trial):
    topK = optuna_trial.suggest_int("topK", 290, 320)
    l1 = optuna_trial.suggest_float("l1_ratio", 0.1, 0.6)
    alpha = optuna_trial.suggest_float("alpha", 0.0001, 0.001)

    
    recommender = slim_model(URM_train)
    recommender.fit(**optuna_trial.params)
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender)
    
    # Save MAP alongside RECALL
    trial_results = {
        "RECALL": eval_res["RECALL"][10],
        "MAP": eval_res["MAP"][10]  # Save MAP for reference
    }
    
    # Save the MAP value to the optuna trial user attributes for later use
    optuna_trial.set_user_attr("MAP", trial_results["MAP"])
    
    # Return RECALL for optimization
    return trial_results["RECALL"]

In [27]:
class SaveResults(object):
    def __init__(self, file_path=None):
        self.file_path = file_path
        self.results = []
    
    def __call__(self, optuna_study, optuna_trial):
        trial_result = optuna_trial.params.copy()
        trial_result["iteration"] = optuna_trial.number
        trial_result["RECALL"] = optuna_trial.value
        
        # Retrieve the MAP metric from user attributes
        trial_result["MAP"] = optuna_trial.user_attrs.get("MAP")
        
        self.results.append(trial_result)
        
        # Convert to DataFrame and save to CSV
        results_df = pd.DataFrame(self.results)
        results_df.to_csv(self.file_path, index=False)


In [28]:
study = op.create_study(direction="maximize")
save_results= SaveResults("result_experiments/SLIM/results_optuna_recall.csv")
study.optimize(objective, callbacks=[save_results], n_trials=100)

[I 2024-12-26 17:26:49,968] A new study created in memory with name: no-name-6a5b9680-5a0f-4560-9d93-02d3fa44e82e


SLIMElasticNetRecommender: Processed 38121 (100.0%) in 4.31 min. Items per second: 147.49
EvaluatorHoldout: Processed 34728 (100.0%) in 8.26 sec. Users per second: 4202


[I 2024-12-26 17:31:16,774] Trial 0 finished with value: 0.09159165610964941 and parameters: {'topK': 312, 'l1_ratio': 0.31883575203896664, 'alpha': 0.0009592179174882873}. Best is trial 0 with value: 0.09159165610964941.


SLIMElasticNetRecommender: Processed 30864 (81.0%) in 5.00 min. Items per second: 102.88
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 6.07 min. Items per second: 104.74
EvaluatorHoldout: Processed 34728 (100.0%) in 8.70 sec. Users per second: 3991


[I 2024-12-26 17:37:29,449] Trial 1 finished with value: 0.10823893732658425 and parameters: {'topK': 305, 'l1_ratio': 0.35737699464954387, 'alpha': 0.0005506592354802661}. Best is trial 1 with value: 0.10823893732658425.


...

[I 2024-12-28 11:49:45,942] Trial 97 finished with value: 0.12754608482837934 and parameters: {'topK': 291, 'l1_ratio': 0.12820042515302368, 'alpha': 0.0003705846557441874}. Best is trial 85 with value: 0.12847829991255766.


SLIMElasticNetRecommender: Processed 22986 (60.3%) in 5.00 min. Items per second: 76.61
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 8.10 min. Items per second: 78.42
EvaluatorHoldout: Processed 34728 (100.0%) in 8.85 sec. Users per second: 3922


[I 2024-12-28 11:58:00,926] Trial 98 finished with value: 0.11372208394523835 and parameters: {'topK': 290, 'l1_ratio': 0.19565480762560777, 'alpha': 0.0007822293247459054}. Best is trial 85 with value: 0.12847829991255766.


SLIMElasticNetRecommender: Processed 12892 (33.8%) in 5.00 min. Items per second: 42.97
SLIMElasticNetRecommender: Processed 27180 (71.3%) in 10.00 min. Items per second: 45.30
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 13.69 min. Items per second: 46.42
EvaluatorHoldout: Processed 34728 (100.0%) in 9.89 sec. Users per second: 3511


[I 2024-12-28 12:11:52,028] Trial 99 finished with value: 0.1246511282236462 and parameters: {'topK': 297, 'l1_ratio': 0.33625123585063454, 'alpha': 0.0002505032311320224}. Best is trial 85 with value: 0.12847829991255766.


,topK,l1_ratio,alpha,result
0,720.0,0.343464,0.768518,0.000265
1,145.0,0.930758,0.587530,0.000265
2,386.0,0.106288,0.872912,0.000265
3,26.0,0.543890,0.859017,0.000265


In [3]:
old_results = pd.read_csv("result_experiments/SLIM/results.csv")

In [5]:
old_results.sort_values(by=['MAP'], ascending=False)

,topK,l1_ratio,alpha,time,cutoff,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
19,300,0.11800,0.00026,2024-11-30 07:54:47,10,0.111556,0.166356,0.141104,0.060479,0.087965,...,0.994991,0.569622,0.994991,0.240193,13.452813,0.999874,0.395067,0.911396,1.359627,0.243844
63,330,0.11100,0.00026,2024-12-01 06:51:24,10,0.111536,0.166336,0.141116,0.060466,0.087930,...,0.994991,0.569398,0.994991,0.240864,13.455295,0.999873,0.396171,0.911564,1.363084,0.243832
55,320,0.11625,0.00026,2024-12-01 04:02:42,10,0.111522,0.166355,0.141133,0.060464,0.087947,...,0.994991,0.569510,0.994991,0.240054,13.450461,0.999873,0.394839,0.911236,1.364541,0.243797
39,310,0.11800,0.00026,2024-11-30 22:22:20,10,0.111508,0.166290,0.141070,0.060463,0.087942,...,0.994991,0.569538,0.994991,0.239982,13.450652,0.999873,0.394719,0.911249,1.362562,0.243814
43,320,0.11100,0.00026,2024-11-30 23:49:08,10,0.111528,0.166322,0.141103,0.060460,0.087929,...,0.994991,0.569398,0.994991,0.241075,13.457495,0.999874,0.396518,0.911713,1.359875,0.243864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,350,0.11100,0.00020,2024-12-02 03:04:27,10,0.111157,0.165720,0.140639,0.060305,0.087739,...,0.994991,0.568278,0.994991,0.253919,13.534857,0.999886,0.417643,0.916954,1.326083,0.244530
20,310,0.11100,0.00020,2024-11-30 08:15:55,10,0.111084,0.165681,0.140623,0.060299,0.087732,...,0.994991,0.568027,0.994991,0.255103,13.545145,0.999889,0.419591,0.917651,1.312987,0.244680
60,330,0.11100,0.00020,2024-12-01 05:48:24,10,0.111120,0.165680,0.140613,0.060298,0.087725,...,0.994991,0.568167,0.994991,0.254458,13.539355,0.999887,0.418530,0.917259,1.320494,0.244595
40,320,0.11100,0.00020,2024-11-30 22:44:29,10,0.111103,0.165689,0.140637,0.060292,0.087724,...,0.994991,0.568223,0.994991,0.254819,13.542520,0.999888,0.419124,0.917473,1.316684,0.244639


In [7]:
old_results.sort_values(by=['RECALL'], ascending=False)

,topK,l1_ratio,alpha,time,cutoff,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
75,330,0.11625,0.00026,2024-12-01 11:03:56,10,0.111503,0.166348,0.141143,0.060451,0.087932,...,0.994991,0.569510,0.994991,0.239853,13.448351,0.999872,0.394507,0.911093,1.367301,0.243768
55,320,0.11625,0.00026,2024-12-01 04:02:42,10,0.111522,0.166355,0.141133,0.060464,0.087947,...,0.994991,0.569510,0.994991,0.240054,13.450461,0.999873,0.394839,0.911236,1.364541,0.243797
103,350,0.11100,0.00026,2024-12-02 04:07:47,10,0.111497,0.166318,0.141127,0.060441,0.087900,...,0.994991,0.569314,0.994991,0.240487,13.451387,0.999872,0.395552,0.911299,1.368347,0.243777
63,330,0.11100,0.00026,2024-12-01 06:51:24,10,0.111536,0.166336,0.141116,0.060466,0.087930,...,0.994991,0.569398,0.994991,0.240864,13.455295,0.999873,0.396171,0.911564,1.363084,0.243832
79,330,0.11800,0.00026,2024-12-01 13:17:13,10,0.111480,0.166309,0.141116,0.060447,0.087918,...,0.994991,0.569482,0.994991,0.239611,13.446661,0.999872,0.394110,0.910979,1.368187,0.243756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,320,0.11100,0.00020,2024-11-30 22:44:29,10,0.111103,0.165689,0.140637,0.060292,0.087724,...,0.994991,0.568223,0.994991,0.254819,13.542520,0.999888,0.419124,0.917473,1.316684,0.244639
80,340,0.11100,0.00020,2024-12-01 13:39:54,10,0.111162,0.165720,0.140635,0.060310,0.087736,...,0.994991,0.568306,0.994991,0.254164,13.537005,0.999887,0.418047,0.917100,1.323202,0.244562
140,370,0.11100,0.00020,2024-12-03 10:19:45,10,0.111148,0.165709,0.140625,0.060311,0.087738,...,0.994991,0.568278,0.994991,0.253398,13.529806,0.999884,0.416786,0.916612,1.333071,0.244459
20,310,0.11100,0.00020,2024-11-30 08:15:55,10,0.111084,0.165681,0.140623,0.060299,0.087732,...,0.994991,0.568027,0.994991,0.255103,13.545145,0.999889,0.419591,0.917651,1.312987,0.244680
